In [1]:
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import glob

In [2]:
def pre_process_tag(tag):
    tag = tag.replace(" ","_")
    tag = tag.replace('"',"")
    tag = tag.lower()
    return tag

In [3]:
train_folder = './data/train/'

with open('./tagging_test/metadata/mapping_conv_topic.train.txt') as f:
    lines = f.readlines()
mapping = {}
for line in tqdm(lines):
    id_,tag = line.strip().split(' "')
    mapping[id_] = pre_process_tag(tag)



data_distribution = np.unique(list(mapping.values()),return_counts=True)
for key,count in zip(data_distribution[0],data_distribution[1]):
    print(f'{key:20} : {count}')
    os.makedirs(f'{train_folder}{key}',exist_ok=True)

100%|██████████| 240/240 [00:00<00:00, 192362.50it/s]

bank_bailout         : 6
budget               : 22
credit_card          : 41
family_finance       : 63
job_benefits         : 54
taxes                : 54


In [4]:
def pre_process_conversation(file_path):
    with open(text_file) as f:
        lines = f.readlines()
    conversation = []
    all_text = ''
    for line in lines:
        tokens = line.strip().split(" ")
        if len(tokens)>4:
            text = " ".join(tokens[3:])
            text = text.replace("[silence]","")
            text = text.replace("[noise]","")
            text = re.sub('[^a-zA-Z ]','',text)
            conversation.append(text)
            conversation.append(tokens[0])
            all_text+=text + ' '
    return all_text,conversation

In [5]:
text_files = glob.glob('./tagging_test/*.txt')
print(len(text_files))

240


In [6]:

dict_ = {"text":[],"class":[],"file_id":[]}
for text_file in tqdm(text_files):
    all_text,conversation = pre_process_conversation(text_file)
    file_id = text_file.split('/')[-1].split(".")[1]
    conversation_topic = mapping[file_id]
    dict_['file_id'].append(file_id)
    dict_['text'].append(all_text)
    dict_['class'].append(conversation_topic)
    with open(f'{train_folder}{conversation_topic}/{file_id}.txt','w') as f:
        f.write(all_text)
    


100%|██████████| 240/240 [00:00<00:00, 270.38it/s]


In [7]:
df = pd.DataFrame(dict_)
df.head()

,text,class,file_id
0,okay so um yes we do keep uh well we started o...,family_finance,2022
1,all right uh weve got a lot of them laughter t...,credit_card,2023
2,hello hey how you doing not bad not bad Whats...,credit_card,2061
3,well um with credit cards is me i i try to kee...,credit_card,2067
4,hi how are you doing uh i couldnt hear you i s...,job_benefits,2085


In [8]:
df.to_csv('./data/data.csv',index=False)